In [11]:
import graphlab

sales = graphlab.SFrame('kc_house_data.gl/')
train_data,test_data = sales.random_split(.8, seed=0)


In [31]:
sales.head()

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
7129300520,2014-10-13 00:00:00+00:00,221900.0,3.0,1.0,1180.0,5650,1,0
6414100192,2014-12-09 00:00:00+00:00,538000.0,3.0,2.25,2570.0,7242,2,0
5631500400,2015-02-25 00:00:00+00:00,180000.0,2.0,1.0,770.0,10000,1,0
2487200875,2014-12-09 00:00:00+00:00,604000.0,4.0,3.0,1960.0,5000,1,0
1954400510,2015-02-18 00:00:00+00:00,510000.0,3.0,2.0,1680.0,8080,1,0
7237550310,2014-05-12 00:00:00+00:00,1225000.0,4.0,4.5,5420.0,101930,1,0
1321400060,2014-06-27 00:00:00+00:00,257500.0,3.0,2.25,1715.0,6819,2,0
2008000270,2015-01-15 00:00:00+00:00,291850.0,3.0,1.5,1060.0,9711,1,0
2414600126,2015-04-15 00:00:00+00:00,229500.0,3.0,1.0,1780.0,7470,1,0
3793500160,2015-03-12 00:00:00+00:00,323000.0,3.0,2.5,1890.0,6560,2,0


In [13]:
#Write a function to compute the Simple Linear Regression weights using the closed form solution
#y = intercept + slope*x
#intercept = sum(y)*sum(x**2)-sum(x)*sum(x*y)/number*sum(x**2) - sum(x)**2
#slope = number*sum(x*y)-sum(x)*sum(y)/number*sum(x**2)-sum(x)**2

def simple_linear_regression(input_feature, output):
    squared_input_feature = input_feature*input_feature
    number = output.size()
    product = input_feature*output
    intercept_numerator = (output.sum()*squared_input_feature.sum())-(input_feature.sum()*product.sum())
    intercept_denominator = (number*squared_input_feature.sum() - (input_feature.sum())*(input_feature.sum()))
    
    intercept = intercept_numerator/intercept_denominator
    
    slope_numerator = (number*product.sum()-input_feature.sum()*output.sum())
    slope_denominator = (number*squared_input_feature.sum() - (input_feature.sum())*(input_feature.sum()))
    
    slope = slope_numerator/slope_denominator

    return(intercept, slope)

In [14]:
#test simple_linear_regression

test_feature = graphlab.SArray(range(5))
test_output = graphlab.SArray(1 + 1*test_feature)
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print "Intercept: " + str(test_intercept)
print "Slope: " + str(test_slope)

Intercept: 1
Slope: 1


In [16]:
#a simple linear regression model for predicting price based on sqft_living

sqft_intercept, sqft_slope = simple_linear_regression(train_data['sqft_living'], train_data['price'])

print "Intercept: " + str(sqft_intercept)
print "Slope: " + str(sqft_slope)

Intercept: -47116.0765749
Slope: 281.958838568


In [21]:
#Write a function that accepts a column of data ‘input_feature’, the ‘slope’
#and the ‘intercept’ you learned, and returns an a column of predictions

def get_regression_predictions(input_feature, intercept, slope):
    predicted_output = intercept + (slope*input_feature)
    
    return(predicted_output)

In [22]:
#6. Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?

my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)

print "The estimated price for a house with %d squarefeet is $%.2f" % (my_house_sqft, estimated_price)

The estimated price for a house with 2650 squarefeet is $700074.85


In [53]:
#Write a function that accepts column of data: ‘input_feature’, and ‘output’ and the regression parameters
#‘slope’ and ‘intercept’ and outputs the Residual Sum of Squares

#RSS = sum((sum(y)-(intercept+slope*input_features))**2)

def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    predicted_output = intercept + (slope*input_feature)
    residual = output-predicted_output
    RSS = (residual*residual).sum()
    
    return(RSS)

In [54]:
#test get_residual_sum_of_squares
#Since the data in the test model lie exactly on a line the residual sum of squares should be zero

print get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope)

0


In [55]:
#8. Quiz Question: According to this function and the slope and intercept from (4) 
#What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?

rss_prices_on_sqft = get_residual_sum_of_squares(train_data['sqft_living'], train_data['price'], sqft_intercept, sqft_slope)

print 'The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft)

The RSS of predicting Prices based on Square Feet is : 1.20191835632e+15


In [29]:
#Write a function that accept a column of data:‘output’ and the regression parameters ‘slope’ and ‘intercept’ 
# and outputs the column of data: ‘estimated_input’

#Do this by solving the linear function output = intercept + slope*input for the ‘input’ variable
#(i.e. ‘input’ should be on one side of the equals sign by itself)

def inverse_regression_predictions(output, intercept, slope):
    estimated_input = (output-intercept)/slope
    
    return(estimated_input)

In [30]:
#10. Quiz Question: According to this function and the regression slope and intercept from (3)
#what is the estimated square-feet for a house costing $800,000?

my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)

print "The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet)

The estimated squarefeet for a house worth $800000.00 is 3004


In [48]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'

bedroom_intercept, bedroom_slope = simple_linear_regression(train_data['bedrooms'], train_data['price'])

print "Intercept: " + str(bedroom_intercept)
print "Slope: " + str(bedroom_slope)

Intercept: 109473.180469
Slope: 127588.952175


In [47]:
import matplotlib.pyplot as plt
%matplotlib inline

graphlab.canvas.set_target('ipynb')
sales.show(view="Scatter Plot", x="bedrooms", y="price")

In [58]:
# Compute RSS when using bedrooms on TEST data:

rss_prices_on_bedrooms = get_residual_sum_of_squares(test_data['bedrooms'], test_data['price'], bedroom_intercept, bedroom_slope)

print 'The RSS of predicting Prices based on Bedrooms is : ' + str(rss_prices_on_bedrooms)

The RSS of predicting Prices based on Bedrooms is : 4.93364582868e+14


In [57]:
# Compute RSS when using squarefeet on TEST data:

rss_prices_on_sqft_test = get_residual_sum_of_squares(test_data['sqft_living'], test_data['price'], sqft_intercept, sqft_slope)

print 'The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft_test)

The RSS of predicting Prices based on Square Feet is : 2.75402936247e+14
